## Classes and Interfaces

### Compose Classes Instead of Nesting Many Levels of Built-in Types

python提供的丰富的数据结构，以及简便的组合方式，使得我们很容易嵌套多层。

比如dict with values that are dicts，这时就应该用namedtuple, 或是自己封装的类来管理

（但我觉得嵌套个两三层都还行

### Accept Functions Instead of Classes for Simple Interfaces


python支持first-class functions

比如函数可以作为参数传递，成员函数可以直接通过函数名访问

> https://stackoverflow.com/questions/27392402/what-is-first-class-function-in-python#:~:text=All%20functions%20in%20Python%20are,(like%20integers%20or%20strings).
A first-class function is not a particular kind of function. All functions in Python are first-class functions. To say that functions are first-class in a certain programming language means that they can be passed around and manipulated similarly to how you would pass around and manipulate other kinds of objects (like integers or strings). You can assign a function to a variable, pass it as an argument to another function, etc. 

因此很多功能可以直接利用python封装好的module, 传入一个自定义的function, 完成一个class的内容。

例如`defaultdict(key)`, `sort(key)`等等。

传递的函数可以是

- lambda
- 普通函数
- 成员函数(通过object.function访问 -> first-class)
- 重载了__call__的类对象

In [2]:
from collections import defaultdict

class BetterCountMissing:
    def __init__(self):
        self.added = 0
    def __call__(self):
        self.added += 1
        return 0

counter = BetterCountMissing()
result = defaultdict(counter, {"a": 2})
increments = {"b": 3}
for k, v in increments.items():
    result[k] += v
print(result)

defaultdict(<__main__.BetterCountMissing object at 0x7f34c102bd90>, {'a': 2, 'b': 3})


### Using `@classmethod` Polymnorphism to Construct Objects Generically

python不支持构造函数的重载，而@classmethod可以用来简接实现构造函数(更像是工厂方法）
 

In [2]:
class Student:
    def __init__(self, name):
        self.name = name
    
    @classmethod
    def from_list(cls, names):
        students = [cls(i) for i in names]
        return students
    
    def __repr__(self):
        return "name: " + self.name
    
names = ['Bob', 'Alice']
students = Student.from_list(names)
print(students)

[name: Bob, name: Alice]


### Initialize Parent Class with `super`

python支持多重继承，如果用`Parent.__init__(self, key)`的方式构造父类，可能会因为调用的顺序不同，或是祖先类构造多次，导致难以发现的bug.

而super().__init__()解决了这一问题，按照mro列表的顺序进行构造函数的调用，并保证每个构造函数只被调用一次。

In [3]:
class Root:
    def __init__(self, v):
        self.v = v

class Increment(Root):
    def __init__(self, v):
        super().__init__(v)
        self.v += 1
        
class Double(Root):
    def __init__(self, v):
        super().__init__(v)
        self.v *= 2

class Func(Increment, Double): # 决定了mro的顺序, "in Python the class hierarchy is defined right to left"
    def __init__(self, v):
        super().__init__(v)

        
func = Func(5)
print(Func.mro())
print(func.v) # double先递归结束, 即先+1, 再*2

[<class '__main__.Func'>, <class '__main__.Increment'>, <class '__main__.Double'>, <class '__main__.Root'>, <class 'object'>]
11


### Consider Composing Functionality with Mix-in Classes

> "Python supports a simple type of multiple inheritance which allows the creation of Mixins. Mixins are a sort of class that is used to "mix in" extra properties and methods into a class. This allows you to create classes in a compositional style."

mixin很像是java的interface, 只不过python支持多重继承，因此可以直接inhert mix-xin class而不是extends.

mixin不会实现构造函数，没有instance attribute(即成员变量)
```
python分为class attribute(即静态变量)和instance attribute(即成员变量, 例如self.variable)
```
mixin的应用场景有以下几种

- 多个class共用一个功能
- 将相关的功能整合在一起，通过多个mixin的组合，派生出一个可以实现复杂功能的class

### Prefer Public Attributes Over Private Ones

> "We are all consenting adults here"

python没有严格约束变量的可见性，即使是private变量也可以通过_class__attribute访问

大部分情况下都推荐使用protected而不是private

唯一需要考虑使用private的情况是, "when you'are worried about nameing conflicts with subclasses", 即派生类定义了同名变量（因为派生类往往是out of your control的)

为了减少这种问题出现的概率，就可以使用private attribute.

### Inherit from `collections.abc` for Custom Container Types

abs: abstract base class
    
- 如果要实现简单的容器类，可以直接继承list
    
- 如果要自己实现复杂的容器类，可以继承`collecitons.abc`。
  利用抽象函数的性质，保证不会遗漏必须的成员函数重载。